## ⚙️ Setup

### Configure API Key and Import Dependencies

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in .env file")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key setup complete.


In [2]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# Configure Retry Options
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry configuration set.")

✅ Retry configuration set.


## 🔍 Section 1: Researcher Agent (Specialist Tool)

This agent conducts comprehensive research on a lead, using Google Search to find:
- Professional background and career history
- Company and role information
- Social media presence
- Recent achievements or projects
- Industry involvement

The main agent will call this as a tool to gather intelligence before crafting the outreach email.

In [4]:
researcher_agent = LlmAgent(
    name="researcher_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a comprehensive lead researcher. Your job is to investigate a potential business contact and gather detailed intelligence.
    
    For the given lead name and context:
    1. Use `google_search()` to find professional information about the lead
    2. Search for their role, company, and career background
    3. Look for their social media presence (LinkedIn, Twitter, GitHub, etc.)
    4. Find recent projects, achievements, or publications they're involved with
    5. Identify their industry focus and expertise areas
    
    Provide a structured research summary that includes:
    - Name and current/recent role
    - Company or organization
    - Professional background (3-5 key points)
    - Social media/online presence
    - Recent achievements or projects
    - Industry focus and expertise areas
    - Any notable connections or activities
    
    Be thorough but concise. Focus on information that will help craft a personalized outreach email.
    """,
    tools=[google_search],
)

print("✅ Researcher agent created")
print("🔧 Tools available:")
print("  • google_search - Comprehensive web search for lead information")

✅ Researcher agent created
🔧 Tools available:
  • google_search - Comprehensive web search for lead information


## 📧 Section 2: Main Outreach Agent (Uses Researcher as Tool)

This is the orchestrator agent that:
1. Takes career field and lead name as input
2. Calls the researcher agent to gather intelligence
3. Uses that research to craft a highly personalized cold outreach email

This demonstrates the **Agent Tool** pattern: using another agent as a tool within a main agent.

In [5]:
main_outreach_agent = LlmAgent(
    name="main_outreach_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a personalized cold outreach specialist. Your role is to create highly targeted, personalized emails to potential leads.
    
    You have two critical tasks:
    
    **Task 1: Research the Lead**
    Use the researcher_agent tool to gather comprehensive information about the lead. Pass the lead's name and career context to the researcher.
    
    **Task 2: Craft Personalized Outreach Email**
    Based on the researcher's findings, create a cold outreach email that:
    - Opens with a personalized reference to something specific about them (recent project, achievement, company, or expertise)
    - Demonstrates genuine knowledge of their background
    - Explains a clear value proposition relevant to THEIR work and interests
    - Is professional, concise, and authentic (no generic templates)
    - Includes a clear call-to-action
    - Is between 100-150 words
    
    **Email Format:**
    Subject: [Create a compelling subject line]
    
    Body:
    [Personalized email content]
    
    **Important:** Your research findings should be visible in your email. For example, if research shows they work at a specific company or have a specific role, reference it directly.
    """,
    tools=[AgentTool(agent=researcher_agent)],
)

print("✅ Main outreach agent created")
print("🤖 Agent Tool configuration:")
print("  • researcher_agent - Delegates lead research")
print("\n🎯 Orchestration pattern:")
print("  1. Main agent receives: career field + lead name")
print("  2. Main agent calls: researcher_agent (as tool)")
print("  3. Main agent uses: research results → personalized email")

✅ Main outreach agent created
🤖 Agent Tool configuration:
  • researcher_agent - Delegates lead research

🎯 Orchestration pattern:
  1. Main agent receives: career field + lead name
  2. Main agent calls: researcher_agent (as tool)
  3. Main agent uses: research results → personalized email


## 🚀 Section 3: Running the Workflow

Set your inputs below and run the agent workflow:

In [14]:
# ========== USER INPUTS ==========
# Modify these variables with your own values

YOUR_CAREER_FIELD = "AI/ML Engineer specializing in Agent Development"
LEAD_NAME = "Elvis Warutumo"

print(f"🎯 Workflow Configuration:")
print(f"   Career Field: {YOUR_CAREER_FIELD}")
print(f"   Lead Name: {LEAD_NAME}")
print(f"\n🔄 Executing workflow...\n")

🎯 Workflow Configuration:
   Career Field: AI/ML Engineer specializing in Agent Development
   Lead Name: Elvis Warutumo

🔄 Executing workflow...



In [15]:
# Create runner and execute the workflow
runner = InMemoryRunner(agent=main_outreach_agent)

# Craft the user prompt with career context
user_prompt = f"""I work in {YOUR_CAREER_FIELD}. 

Please research {LEAD_NAME} and create a personalized cold outreach email for them. 
Make the email relevant to my career field and their background. 
The email should be authentic and demonstrate that I've done genuine research on them."""

# Run the agent
response = await runner.run_debug(user_prompt)


 ### Created new session: debug_session_id

User > I work in AI/ML Engineer specializing in Agent Development. 

Please research Elvis Warutumo and create a personalized cold outreach email for them. 
Make the email relevant to my career field and their background. 
The email should be authentic and demonstrate that I've done genuine research on them.
main_outreach_agent > Subject: Enhancing AI Agent Development with Scalable Solutions

Hi Elvis,

I came across your work in AI/ML engineering, particularly your specialization in agent development and your role as co-founder of DAIR.AI. Your PhD in NLP and experience with LLMs at Meta AI are truly impressive, especially your contributions to Galactica.

Given your background and current focus on practical AI applications and education, I believe you'd be interested in [mention a specific area relevant to his work, e.g., novel approaches to LLM scalability, advanced agent architectures, or efficient prompt engineering strategies]. My wor

## 📊 Section 4: Understanding the Workflow

### What Happened?

**Step 1: Research Phase**
- The main agent called the researcher agent as a tool
- The researcher used Google Search to find information about the lead
- Results were returned to the main agent

**Step 2: Personalization Phase**
- The main agent received the research findings
- It used those findings to craft a personalized cold outreach email
- The email references specific details from the research

### Why This Pattern is Powerful

1. **Separation of Concerns** - Researcher focuses on finding data, main agent focuses on crafting messages
2. **Reusability** - The researcher agent can be used by other agents for different purposes
3. **Scalability** - Easy to add more specialist agents (copywriter, sales expert, industry specialist)
4. **Quality** - Each agent is optimized for its specific task

### Comparison with Other Patterns

| Pattern | Use Case | Example |
|---------|----------|----------|
| **Agent Tool** (used here) | Delegation within workflow | Researcher as tool for outreach |
| **Sequential Agents** | Ordered processing | Finder → Solver → Outreach |
| **Parallel Agents** | Concurrent research | Reddit + Quora + Freelance simultaneous |
| **Loop Agents** | Iterative refinement | Critic loops for quality |

## 💡 Section 5: Customization Guide

### How to Customize This Workflow

#### Option 1: Change Lead Information
```python
YOUR_CAREER_FIELD = "Data Science / Product Analytics"
LEAD_NAME = "Sheryl Sandberg"
```

#### Option 2: Add More Specialized Agents
```python
# Add a copywriter agent for tone customization
copywriter_agent = LlmAgent(
    name="copywriter_agent",
    instruction="Convert emails to [tone: professional/casual/urgent]",
    tools=[]
)

# Add as tool to main agent
main_agent.tools = [AgentTool(researcher_agent), AgentTool(copywriter_agent)]
```

#### Option 3: Use Custom Function Tools
```python
def validate_email_format(email_text: str) -> dict:
    """Validate email structure and compliance"""
    return {"status": "valid", "issues": []}

main_agent.tools = [
    AgentTool(researcher_agent),
    validate_email_format  # Function tool
]
```

#### Option 4: Enhance Research with Additional Tools
```python
from google.adk.tools import web_crawl, twitter_search

researcher_agent.tools = [
    google_search,
    web_crawl,  # Get full webpage content
    twitter_search  # Find recent tweets
]
```

## 🔌 Section 6: Integration with Fiona

### How This Agent Fits Into Fiona's Email Campaign Platform

This personalized lead outreach agent extends Fiona by automating intelligent prospect research and email generation:

**Workflow Integration:**
1. **Input**: User provides career field and lead name via Fiona UI
2. **Processing**: Agent research + email generation (this notebook)
3. **Output**: Generated email → Fiona's email template library
4. **Action**: User reviews and sends via Fiona's campaign system

**Benefits for Fiona Users:**
- **Speed**: Personalized emails in seconds instead of hours
- **Quality**: Research-backed personalization increases open rates
- **Scale**: Generate dozens of personalized emails for different leads
- **Authenticity**: Research-based approach avoids generic templates

### Backend Integration Points
```
Fiona Frontend
    ↓
[Enter: Career Field + Lead Name]
    ↓
This Agent Notebook
    ↓
Generated Email + Research Summary
    ↓
Fiona Email Template Library
    ↓
Send via Fiona Campaigns
```

## ✅ Summary

You've successfully built a sophisticated multi-agent system that demonstrates:

✅ **Agent Tools Pattern** - Using agents as tools within other agents  
✅ **Research Delegation** - Specialist agent for comprehensive lead research  
✅ **Personalization** - Using research findings to create authentic outreach  
✅ **Fiona Integration** - How this fits into an email campaign platform  

### Key Takeaways

1. **Orchestration** - Main agents can coordinate specialist agents as tools
2. **Research-Backed** - Real web research ensures authentic personalization
3. **Scalable** - Easy to add more agents or tools as needed
4. **Production-Ready** - This pattern works for real cold outreach workflows

### Next Steps

- Test with different leads and career fields
- Add email tone customization (formal/casual/urgent)
- Create follow-up email agents
- Integrate with Fiona backend for end-to-end automation